# Setup

In [1]:
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import requests
import tkinter as tk
from tkinter import font as tkfont

from OpenWeather import *

W = OpenWeather(api_key, city, latlong)
W.initial_load()
lastUpdate = datetime.time

OpenWeather instance created for Seattle, (47.6894, -122.406).
data loaded


In [260]:
# Colors
EIGENGRAU = '#16161d'
DEFAULT_PRIMARY = '#6eb32e'#cef1dd
DEFAULT_SECONDARY = 'gray'
DEFAULT_TERTIARY = 'pink'

warmedgreen = '#dd8c13'
cooledgreen = '#e1ecf5'

## Support Classes

In [261]:
class Tab_43(tk.Canvas) :
    ## 215X42 DISPLAY TAB
    def __init__(self, master, label = 'label', barcolor = 'green', textcolor = 'green', **kwargs) :
        tk.Canvas.__init__(self, master = master, bg = 'black', 
                           width = 215, height = 43,
                           highlightthickness = 0,
                           highlightbackground = 'black', highlightcolor = 'black')
        
        bar = self.create_rectangle(22, 1, 214, 42, 
                                    outline = barcolor, fill = barcolor)
        cap = self.create_arc(1, 1, 42, 42, 
                              start = 90, extent = 180,
                              fill = barcolor, outline = barcolor)
        
        valuelabel = tk.Label(self, anchor = 'e', 
                              bg = barcolor, fg = 'black', 
                              text = label, font = tkfont.Font(family = 'Helvetica', size=10, weight = 'bold'))
        valuelabel.place(anchor = 'e', x = 77, y = 35)
        
        value = tk.Label(self, anchor = 'e', padx = 5,
                         bg = 'black', fg = textcolor,
                         width = 4, justify = 'right',
                         text = '100.9', font = tkfont.Font(family="Helvetica",size=36))
        value.place(anchor = 'e', x = 204, rely = 0.5)
    
class Tab_70(tk.Canvas) :
    ## 350X70 DISPLAY TAB
    def __init__(self, master, label = 'label', barcolor = 'green', textcolor = 'green', **kwargs) :
        tk.Canvas.__init__(self, master = master, bg = 'black', 
                           width = 350, height = 70,
                           highlightthickness = 0,
                           highlightbackground = 'black', highlightcolor = 'black')
        
        bar = self.create_rectangle(36, 1, 349, 69, 
                                    outline = barcolor, fill = barcolor)
        
        cap = self.create_arc(1, 1, 69, 69, 
                              start = 90, extent = 180,
                              fill = barcolor, outline = barcolor)
        
        valuelabel = tk.Label(self, anchor = 'e', 
                              bg = barcolor, fg = 'black', 
                              text = label, font = tkfont.Font(family = 'Helvetica', size = 12, weight = 'bold'))
        valuelabel.place(anchor = 'e', x = 135, y = 58)
        
        value = tk.Label(self, anchor = 'e', padx = 5,
                         bg = 'black', fg = textcolor,
                         width = 4, justify = 'right',
                         text = '99.9', font = 'Helvetica, 60')
        value.place(anchor = 'e', x = 339, rely = 0.5)

# GUI

In [269]:
class GUI(tk.Tk) :
    def __init__(self, name = 'root', *args, **kwargs) :
        tk.Tk.__init__(self, *args, **kwargs)
        
        self.title = 'ParkerTron GUI WIP'
        self.geometry('800x480')
        self.resizable(False, False)
        
        container = tk.Frame(self, name = 'root')
        container.pack(side = 'top', fill = 'both', expand = True)
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
        
        self.frames = {}
        # add each page (class name?) into the list; object name, not string
        frame_list = [Weather]
        for F in frame_list:
            page_name = F.__name__
            frame = F(parent = container, controller = self)
            self.frames[page_name] = frame
            frame.grid(row = 0, column = 0, sticky = 'nsew')
        
        self.show_frame('Weather')
        
    def show_frame(self, page_name) :
        '''Show a frame for the given page name'''
        frame = self.frames[page_name]
        frame.tkraise()
    
class Weather(tk.Frame) :
    def __init__(self, parent, controller, name = 'weather', **kwargs) :
        tk.Frame.__init__(self, parent, name = name, bg = 'black')
        self.controller = controller
        self.title_text = 'Weather' 
        
        # Dimensions
        width = 800
        height = 480
        navwidth = 185
        fieldwidth = width - navwidth
        topheight = 70
        bottomheight = 43
        navheight = height - topheight - bottomheight
        
        # Navigation Bar
        nav = tk.Frame(self, bg = '#6eb32e', width = navwidth, height = navheight)
        nav.place(anchor = 'nw', x = 0 + 2, y = topheight)
        
        # Top Bar
        topbar = tk.Frame(self, bg = '#6eb32e', width = 617, height = topheight)
        topbar.place(anchor = 'nw', x = navwidth, y = 0 + 2)
        toplabel = tk.Label(topbar, bg = 'black', fg = '#cef1dd', padx = 10, pady = 10,
                            text = 'ParkerTron Beta', font = 'Orbitron, 36', justify = 'right')
        toplabel.place(anchor = 'e', x = fieldwidth - 27, rely = .5)

        #Bottom Navigation Bar
        bottombar = tk.Frame(self, bg = '#6eb32e', width = 617, height = bottomheight)
        bottombar.place(anchor = 'sw', x = navwidth, y = height - 2)
        bottomlabel = tk.Label(bottombar, bg = '#6eb32e', fg = '#cef1dd', text = 'Bottom Navigation Bar')
        bottomlabel.place(anchor = 'center', relx = 0.5, rely = 0.5)

        # Main Display Field
        field = tk.Frame(self, bg = 'black', width = fieldwidth, height = navheight)
        field.place(anchor = 'nw', x = navwidth, y = topheight)
        
        currenttext = f'Current conditions as of {W.current.index[0]}\n  {W.current_conditions_brief}, {W.current_conditions}'
        currentconditions = tk.Label(field, bg = 'black', fg = '#cef1dd',
                                    text = currenttext, font = 'Helvetica, 22', justify = 'left')
        currentconditions.place(anchor = 'nw', x = 0, y = 0)
        
        # Temperatures
        temperatureframe = tk.Frame(field, name = 'temperatureframe', bg = 'black', width = 350, height = 170)
        temperatureframe.place(anchor = 'e', x = 600, rely = 0.5)

        current = Tab_70(temperatureframe, name = 'currenttemp', label = 'current', barcolor = DEFAULT_PRIMARY, textcolor = DEFAULT_PRIMARY)
        current.place(anchor = 'ne', x = 350, y = 0)

        hi = Tab_43(temperatureframe, name = 'hitemp', label = 'hi', barcolor = warmedgreen, textcolor = warmedgreen)
        hi.place(anchor = 'ne', x = 350, y = 75)

        lo = Tab_43(temperatureframe, name = 'lotemp', label = 'lo', barcolor = cooledgreen, textcolor = cooledgreen)
        lo.place(anchor = 'ne', x = 350, y = 123)
        
# Begin main loop of application
if __name__ == '__main__' :
    app = GUI('800x480')    
    #app.overrideredirect(1)
    app.mainloop()

# TESTING BELOW

In [270]:
import tkinter as tk
from tkinter import font as tkfont

main = tk.Tk()
main.title('ParkerTron')

#width = main.winfo_screenwidth()
#height = main.winfo_screenheight()
width = 800
height = 480
main.geometry('800x480') # not a best practice to hard code sizes?
main.resizable(False, False)

canvas = tk.Canvas(master = main, 
                   bd = 0, relief = 'flat',
                   highlightthickness = 0,
                   bg = 'black', width = width, height = height)
canvas.pack()

temperatureframe = tk.Frame(main, name = 'temperatureframe', bg = 'black', width = 350, height = 170)
temperatureframe.place(anchor = 'center', relx = 0.5, rely = 0.5)

current = Tab_70(temperatureframe, name = 'currenttemp', label = 'current', barcolor = DEFAULT_PRIMARY, textcolor = DEFAULT_PRIMARY)
current.place(anchor = 'ne', x = 350, y = 0)

hi = Tab_43(temperatureframe, name = 'hitemp', label = 'hi', barcolor = warmedgreen, textcolor = DEFAULT_PRIMARY)
hi.place(anchor = 'ne', x = 350, y = 75)

lo = Tab_43(temperatureframe, name = 'lotemp', label = 'lo', barcolor = cooledgreen, textcolor = DEFAULT_PRIMARY)
lo.place(anchor = 'ne', x = 350, y = 123)

main.mainloop()

In [ ]:
import datetime
import time
import pandas as pd
import requests

# Variables
api_key = '06b639d0c99a00d538d0ddee99e33617'
api_key2 = 'bd627fefaef37e87a1709da33a1d8bd1'
city = 'Seattle'
zipcode = '98117'
latlong = (47.6894, -122.4060)     # ballard

class OpenWeather :
    """
    Object wrapper for the OpenWeatherData API.
    """
    def __init__(self, api_key, city, latlong):        
        self.appid = api_key
        self.city = city
        self.latlong = latlong
        self.lat = str(latlong[0])
        self.lon = str(latlong[1])
        
        self.current = pd.DataFrame()
        self.minutely = pd.DataFrame()
        self.hourly = pd.DataFrame()
        self.daily = pd.DataFrame()
        self.alerts = pd.DataFrame()
        
        # make these into their own classes?
        self.airpollution = pd.DataFrame()
        self.airpollution_forecast = pd.DataFrame()
        
        self.aqi_map = {1 : "good", 2 : "fair", 3 : "moderate", 4 : "poor", 5 : "very poor"}
        
        self.debug = False
        
        print('OpenWeather instance created for {}, {}.'.format(self.city, self.latlong))  
        
    @property
    def appid(self) :
        return self._appid
    
    @appid.setter
    def appid(self, value) :
        self._appid = value
        
    @property
    def city(self) :
        return self._city
    
    @city.setter
    def city(self, value) :
        self._city = value
        
    @property
    def latlong(self):
        return self._latlong
    
    @latlong.setter
    def latlong(self, value) :
        self._latlong = value
        
    @property
    def lat(self) :
        return self._latlong[0]
    
    @lat.setter
    def lat(self, value) :
        self._lat = value
    
    @property
    def lon(self):
        return self._latlong[1]
    
    @lon.setter
    def lon(self, value) :
        self._lon = value
        
    @property
    def current_temperature(self) :
        return self.current.temp[0]
    
    @property
    def lo_temp(self) :
        return self.daily.temp[0]['min']
    
    @property
    def hi_temp(self) :
        return self.daily.temp['max']
        
    # functions
    def fmt_unix_date(self, dt) :
        fmt_string = '%Y-%m-%d %I:%M %p'
        timestamp = datetime.datetime.utcfromtimestamp(dt).strftime(fmt_string)
        return timestamp
    
    def get_onecall(self) :
        # api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}
        try :
            url = 'https://api.openweathermap.org/data/2.5/onecall'
            params = {'lat' : self.lat, 'lon' : self.lon, 'appid' : self.appid, 'units' : 'imperial'}
            response = requests.get(url, params = params)
            
            if response.status_code == 200 :
                #print('Successful request, code: {}'.format(response.status_code))
                return response.json()
                #self.data = response.json()
            else :
                print('HTTP error, code: {}'.format(response.status_code))
                return None
        except :
            print('Error contacting remote server.')
            return None

    def initial_load(self) :
    	self.refresh_onecall()
    	self.airpollution = self.get_airpollution()
    	print('data loaded')

    def parse_onecall(self, data) :
    	# This function takes the returned JSON data and breaks the data into
    	# separate dataframes. Datetime fields are cleaned up and adjusted for
    	# time zone differences.
        if data is None :
            print('No data!')
            return None

        tz_offset = data['timezone_offset']

        current = pd.DataFrame.from_dict(data['current'], orient = 'index').T
        minutely = pd.DataFrame.from_dict(data['minutely'], orient = 'columns')
        hourly = pd.DataFrame.from_dict(data['hourly'], orient = 'columns')
        daily = pd.DataFrame.from_dict(data['daily'], orient = 'columns')

        for df in [current, minutely, hourly, daily] :
            df['dt'] += tz_offset
            df['timestamp'] = [self.fmt_unix_date(d) for d in df['dt']]
            df.set_index('timestamp', inplace = True)
            df['month'] = [datetime.datetime.utcfromtimestamp(d).strftime('%b') for d in df['dt']]
            df['day'] = [datetime.datetime.utcfromtimestamp(d).strftime('%d') for d in df['dt']]
            df['weekday'] = [datetime.datetime.utcfromtimestamp(d).strftime('%a') for d in df['dt']]
        #print('parsed')

        #return current, hourly, daily
        self.current = current
        self.minutely = minutely
        self.hourly = hourly
        self.daily = daily
    
    def refresh_onecall(self) :
        self.parse_onecall(self.get_onecall())
        
    # AIR POLLUTION ----------------------------------------------
    @property
    def aqi(self) :
        return airPollution["list"][0]["main"]["aqi"]
    
    @property
    def aq_components(self) :
        return airPollution["list"][0]["components"]
    
    @property
    def carbon_monoxide(self) :
        return airPollution["list"][0]["components"]["co"]
    
    @property
    def ozone(self) :
        return airPollution["list"][0]["components"]["o3"]
    
    @property
    def suphur_dioxide(self) :
        return airPollution["list"][0]["components"]["s02"]
    
    @property
    def fine_particles(self) :
        return airPollution["list"][0]["components"]["pm2_5"]
    
    @property
    def coarse_particles(self) :
        return airPollution["list"][0]["components"]["pm10"]
    
    def get_airpollution(self) :        
        try :
            url = 'https://api.openweathermap.org/data/2.5/air_pollution'
            params = {'lat' : self.lat, 'lon' : self.lon, 'appid' : self.appid}
            response = requests.get(url, params = params)
            
            if response.status_code == 200 :
                #print('Successful request, code: {}'.format(response.status_code))
                return response.json()
            else :
                print('HTTP error, code: {}'.format(response.status_code))
                print(response.text)
                return None
        except :
            print('Error contacting remote server.')
            return None
    
    def get_airpollution_forecast(self) :
        try :
            url = 'https://api.openweathermap.org/data/2.5/airpollution/forecast'
            params = {'lat' : self.lat, 'lon' : self.lon, 'appid' : 'bd627fefaef37e87a1709da33a1d8bd1'}
            response = requests.get(url, params = params)
            
            if response.status_code == 200 :
                #print('Successful request, code: {}'.format(response.status_code))
                return response.json()
            else :
                print('HTTP error, code: {}'.format(response.status_code))
                return None
        except :
            print('Error contacting remote server.')
            return None
    
    def parse_airpollution_forecast(self) :
        if data is None :
            print('No data!')
            return None
        
    # HISTORICAL WEATHER -----------------------------------------
    def get_historical_weather(self) :
        # date from the past 5 days, Unix time, UTC time zone
        dt = ''
        try :
            url = 'https://api.openweathermap.org/data/2.5/onecall/timemachine'
            params = {'lat' : self.lat, 'lon' : self.lon, 'dt' : dt, 'appid' : 'bd627fefaef37e87a1709da33a1d8bd1'}
            response = requests.get(url, params = params)
            
            if response.status_code == 200 :
                #print('Successful request, code: {}'.format(response.status_code))
                return response.json()
            else :
                print('HTTP error, code: {}'.format(response.status_code))
                return None
        except :
            print('Error contacting remote server.')
            return None
    
    def parse_historical_weather(self, data) :
        if data is None :
            print('No data!')
            return None